# Team Update 6

For this assignment, you'll be looking at the current state of your design process and evaluating the effect of potential changes to your design.

In [ ]:
import os
from pathlib import Path

import sys
ISST_DIR = str(Path(os.getcwd()).parent.parent.parent)
sys.path.append(ISST_DIR)

import numpy as np
import arviz as az
import pymc as pm

import matplotlib.pyplot as plt

import ISST

# Evaluating Utility

In Team Update 5, you implemented your Impact Tables, and looked at the effects of individual risks on the parameters that those Impact Tables quantified. Now we'll start to work on actually using the results of the analysis for decision making in our design process. To do this, we'll need to revisit utility functions.

In Team Update 5, you plotted the discrete and logistic utility functions defined by your Impact Tables, but you also have the option of defining your own utility functions.

For instance, if your impact parameters is some "error" from a nominal value, e.g. you're targeting a particular orbital altitude, you could define an Impact table that defines the utility function in terms the deviation from the nominal value: 

In [ ]:
Altitude_Impact_Table = ISST.ImpactTable(name='Orbital Altitude Error',
                                         units='km',
                                         utility_breakpoints=[0, 5., 10., 20., 25., 50.],
                                         utilities=[0., -1., -3., -5., -7., -9.],
                                         utility_names=['0 km Error',
                                                        '5 km Error',
                                                        '10 km Error',
                                                        '20 km Error',
                                                        '25 km Error',
                                                        '50 km Error']
)

Altitude_Impact_Table.plot_utilities()

Let's set up a basic design system incorporating this as a technical parameter, using the same cost and schedule tables as in the Example System Setup notebook, and with a risk of navigation system error. Since we're using just the one risk, we'll set its parameters manually instead of reading them in from a file:

In [ ]:
Nav_Risk = ISST.Risk(name='Navigation Risk',
                     baseline_likelihood = 0.9,
                     
                     #Schedule Risk Parameters
                     schedule_risk_minimum_value=1.,
                     schedule_risk_maximum_value=18.,
                     schedule_risk_most_likely_value=3., 
                                                      
                     # Cost Risk Parameterers         
                     cost_risk_minimum_value=np.log10(100000),
                     cost_risk_maximum_value=np.log10(2000000), 
                     cost_risk_most_likely_value=np.log10(500000),
                                                      
                     # Technical Risk Parameters      
                     technical_risk_minimum_values=[-50.],
                     technical_risk_maximum_values=[200.],
                     technical_risk_most_likely_values=[0.]
                     )

Log_Cost_Risk_Table = ISST.ImpactTable(name='LogCost',
                                       units='logEUR',
                                       utility_breakpoints=[5., 6., 6.69897, 7., 7.69897, 8.],
                                       utilities=[-1., -2., -4., -6., -8., -10.],
                                       utility_names=['100k Euros',
                                                      '1M Euros',
                                                      '5M Euros',
                                                      '10M Euros',
                                                      '50M Euros',
                                                      '100M Euros'])

Schedule_Risk_Table = ISST.ImpactTable(name='Schedule',
                                       units='months',
                                       utility_breakpoints=[0., 3., 6., 12., 24., 36., 60., 120.],
                                       utilities=[0., -0.5, -1., -2., -3., -5., -7., -10.],
                                       utility_names=['0 months',
                                                      '3 months',
                                                      '6 months',
                                                      '12 months',
                                                      '24 months',
                                                      '36 months',
                                                      '60 months',
                                                      '120 months'])

Example_Design_System = ISST.DesignSystem(name='Example_Design_System',
                                          model_context=pm.Model(),
                                          risks=[Nav_Risk],
                                          schedule_risk_table = Schedule_Risk_Table,
                                          cost_risk_table = Log_Cost_Risk_Table,
                                          technical_risk_tables = [Altitude_Impact_Table])

results = Example_Design_System.analyze_system()

Now that we have our results, let's look at the impact on altitude specifically:

In [ ]:
az.summary(results,
           var_names=['Total Orbital Altitude'],
           filter_vars='like',
           round_to=2,
           kind='stats')

In [ ]:
az.plot_posterior(results,
                  var_names=['Total Orbital Altitude'],
                  filter_vars='like',
                  hdi_prob=0.95)

This works well enough, but note that in this example, the direction of the error really affects the actual utility impact of the error, and our utility breakpoints are all positive, based on the absolute value of the error. If we check the utility of hte absolute values of our mean and 95% confidence limits, we get: 

In [ ]:
mean = np.asarray(results.posterior.mean()['Total Orbital Altitude Error Impact'])
confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)['Total Orbital Altitude Error Impact'])

print(f'Utility for Average Impact ({mean: .2f} km): {Altitude_Impact_Table.logistic_utility(mean): .2f}')

print(f'Utility for Lower Confidence Bound ({confidence_bounds[0]: .2f} km): {Altitude_Impact_Table.logistic_utility(np.abs(confidence_bounds)[0]): .2f}')

print(f'Utility for Upper Confidence Bound ({confidence_bounds[1]: .2f} km): {Altitude_Impact_Table.logistic_utility(np.abs(confidence_bounds)[1]): .2f}')

On average, we expect to be 22.77 km off from our nominal orbit altitude and lose -6.25 points of utility due to navigation errors, and more at the extremes of our confidence intervals, but note that the impact of the errors is symmetric whether they're positive or negative. Is that actually what we want?

The Lunar Reconnaissance Orbiter has an orbital perigee (technically periselene for the moon) of 20 km, so being 20 km *below* your nominal altitude is a much worse outcome than being 20 km *above* your nominal altitude. Our utility function probably looks more like an assymetric bell curve, which goes to -10 much faster on the negative side of the curve than it does on the positive side of the curve:

In [ ]:
def bell_curve_utility(impact, max_utility=10, center=0., decay_rate = 1.):
    return -max_utility * (1 - np.exp(-(impact-center)**2/(1/decay_rate)))

def orbital_error_utility(altitude_error):
    return ((altitude_error < 0) * bell_curve_utility(altitude_error, decay_rate = 0.01) +
            (altitude_error >= 0) * bell_curve_utility(altitude_error, decay_rate = 0.0001))

In the above cell, `(altitude_error < 0)` returns `True (==1)` if our error is negative, and `False (==0)` otherwise, vice-versa for `(altitude_error >= 0)`. This way, only the version of bell_curve_utility that has the decay_rate appropriate to that side of the curve is used without having to use an if statement. Let's plot it over a range of potential errors to see what it looks like:

In [ ]:
error_range = np.linspace(-50., 200., 1000)
plt.plot(error_range, orbital_error_utility(error_range))
plt.xlabel('Altitude Error (km)')
plt.ylabel('Utility')
plt.show()

Let's instead calculate the utility for our mean and 95% confidence limits using this utility function, this time dropping the absolute value conversion:

In [ ]:
print(f'Utility for Average Impact ({mean: .2f} km): {orbital_error_utility(mean): .2f}')

print(f'Utility for Lower Confidence Bound ({confidence_bounds[0]: .2f} km): {orbital_error_utility(confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound ({confidence_bounds[1]: .2f} km): {orbital_error_utility(confidence_bounds[1]): .2f}')

Now, our utilities match our understanding of the system better - being 22.77 km above our nominal altitude isn't too bad, but being more than 90 km above is a real problem, however being more than 40 km *below* our nominal altitude is a total loss.

# Reviewing Previous Results

First, let's look at your results from Team Update 5 in terms of utility functions. By default, Team Update 5 saved your results as `Analysis Results.nc`. If you have changed the name of the results file, you'll need to change the name of the file in the next cell:

In [ ]:
TU5_Results = az.from_netcdf('Analysis Results.nc')

First, let's look at the cost and schedule utilities. You can either reconstruct the Impact Tables you used in Team Update 5 by copy and pasting below: 

In [ ]:
Cost_Risk_Table = ISST.ImpactTable(name='Cost',
                                   units = '',
                                   utility_breakpoints=[],
                                   utilities=[],
                                   utility_names=[]
                                   )

Schedule_Risk_Table = ISST.ImpactTable(name='Schedule',
                                       units = '',
                                       utility_breakpoints=[],
                                       utilities=[],
                                       utility_names=[]
                                       )

OR by definining custom utility functions here:

In [ ]:
def custom_cost_utility_function(impact):
    
    return 

In [ ]:
def custom_schedule_utility_function(impact):
    
    return

Run ONE the following cells to create an alias to your chosen utility method for cost:

In [ ]:
# Run this cell to use the Impact Table's logistic utility function
cost_utility_function = Cost_Risk_Table.logistic_utility

In [ ]:
# Run this cell to use the Impact Table's discrete utility function
cost_utility_function = Cost_Risk_Table.discrete_utility

In [ ]:
# Run this cell to use your custom utility function
cost_utility_function = custom_cost_utility_function

And run ONE of the following cells to create an alias for your chosen utility method for schedule:

In [ ]:
# Run this cell to use the Impact Table's logistic utility function
schedule_utility_function = Schedule_Risk_Table.logistic_utility

In [ ]:
# Run this cell to use the Impact Table's discrete utility function
schedule_utility_function = Schedule_Risk_Table.discrete_utility

In [ ]:
# Run this cell to use your custom utility function
schedule_utility_function = custom_schedule_utility_function

Now, let's look at the utility for your mean and 95% confidence limits on cost and schedule:

In [ ]:
cost_mean = np.asarray(results.posterior.mean()['Total Cost Impact'])
schedule_mean = np.asarray(results.posterior.mean()['Total Schedule Impact'])

cost_confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)['Total Cost Impact'])
schedule_confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)['Total Schedule Impact'])

In [ ]:
print(f'Utility for Expected Cost Impact: {cost_utility_function(cost_mean): .2f}')

print(f'Utility for Lower Confidence Bound: {cost_utility_function(cost_confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound: {cost_utility_function(cost_confidence_bounds[1]): .2f}')

In [ ]:
print(f'Utility for Expected Schedule Impact: {schedule_utility_function(schedule_mean): .2f}')

print(f'Utility for Lower Confidence Bound: {schedule_utility_function(schedule_confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound: {schedule_utility_function(schedule_confidence_bounds[1]): .2f}')

Repeat this process for each of your technical impacts, copying and pasting the following cells if you have more than three technical parameters (Make sure the name of the parameter matches Team Update 5):

Technical Parameter 1:

In [ ]:
T1_Impact_Table = ISST.ImpactTable(name='',
                                   units = '',
                                   utility_breakpoints=[],
                                   utilities=[],
                                   utility_names=[]
                                   )

In [ ]:
# Run this cell to use the Impact Table's logistic utility function
T1_utility_function = T1_Risk_Table.logistic_utility

In [ ]:
# Run this cell to use the Impact Table's discrete utility function
T1_utility_function = T1_Risk_Table.discrete_utility

In [ ]:
# Run this cell to use your custom utility function
def custom_T1_utility_function(impact):
    
    return

T1_utility_function = custom_T1_utility_function

In [ ]:
T1_mean = np.asarray(results.posterior.mean()[f'Total {T1_Impact_Table.name} Impact'])
T1_confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)[f'Total {T1_Impact_Table.name} Impact'])

print(f'Utility for Expected {T1_Impact_Table.name} Impact: {T1_utility_function(T1_mean): .2f}')

print(f'Utility for Lower Confidence Bound: {T1_utility_function(T1_confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound: {T1_utility_function(T1_confidence_bounds[1]): .2f}')

Technical Parameter 2:

In [ ]:
T2_Impact_Table = ISST.ImpactTable(name='',
                                   units = '',
                                   utility_breakpoints=[],
                                   utilities=[],
                                   utility_names=[]
                                   )

In [ ]:
# Run this cell to use the Impact Table's logistic utility function
T2_utility_function = T2_Risk_Table.logistic_utility

In [ ]:
# Run this cell to use the Impact Table's discrete utility function
T2_utility_function = T2_Risk_Table.discrete_utility

In [ ]:
# Run this cell to use your custom utility function
def custom_T2_utility_function(impact):
    
    return

T2_utility_function = custom_T2_utility_function

In [ ]:
T2_mean = np.asarray(results.posterior.mean()[f'Total {T2_Impact_Table.name} Impact'])
T2_confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)[f'Total {T2_Impact_Table.name} Impact'])

print(f'Utility for Expected {T2_Impact_Table.name} Impact: {T2_utility_function(T2_mean): .2f}')

print(f'Utility for Lower Confidence Bound: {T2_utility_function(T2_confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound: {T2_utility_function(T2_confidence_bounds[1]): .2f}')

Technical Parameter 3:

In [ ]:
T3_Impact_Table = ISST.ImpactTable(name='',
                                   units = '',
                                   utility_breakpoints=[],
                                   utilities=[],
                                   utility_names=[]
                                   )

In [ ]:
# Run this cell to use the Impact Table's logistic utility function
T3_utility_function = T3_Risk_Table.logistic_utility

In [ ]:
# Run this cell to use the Impact Table's discrete utility function
T3_utility_function = T3_Risk_Table.discrete_utility

In [ ]:
# Run this cell to use your custom utility function
def custom_T2_utility_function(impact):
    
    return

T3_utility_function = custom_T3_utility_function

In [ ]:
T3_mean = np.asarray(results.posterior.mean()[f'Total {T3_Impact_Table.name} Impact'])
T3_confidence_bounds = np.asarray(az.hdi(results, hdi_prob=0.95)[f'Total {T3_Impact_Table.name} Impact'])

print(f'Utility for Expected {T3_Impact_Table.name} Impact: {T3_utility_function(T3_mean): .2f}')

print(f'Utility for Lower Confidence Bound: {T3_utility_function(T3_confidence_bounds[0]): .2f}')

print(f'Utility for Upper Confidence Bound: {T3_utility_function(T3_confidence_bounds[1]): .2f}')

# Design Revisions

Now that we have the baseline from Technical Update 5, it's time to evaluate the effect of any design change you want to make. To do this, we'll need to create a new Design System. If you're re-using the same Impact Tables, and the baseline likelihoods of your risks haven't changed, you can just copy and paste from Team Update 5. If you have made a change, be sure to document it in the accompanying individual assignment.

In [ ]:
TU6_Cost_Impact_Table = ISST.ImpactTable(name='Cost',
                                         units='',
                                         utility_breakpoints=[],
                                         utilities=[],
                                         utility_names=[]
                                         )

TU6_Schedule_Impact_Table = ISST.ImpactTable(name='Schedule',
                                             units = '',
                                             utility_breakpoints=[],
                                             utilities=[],
                                             utility_names=[]
                                             )

TU6_T1_Impact_Table = ISST.ImpactTable(name='',
                                       units = '',
                                       utility_breakpoints=[],
                                       utilities=[],
                                       utility_names=[]
                                       )

TU6_T2_Impact_Table = ISST.ImpactTable(name='',
                                       units = '',
                                       utility_breakpoints=[],
                                       utilities=[],
                                       utility_names=[]
                                       )

TU6_T3_Impact_Table = ISST.ImpactTable(name='',
                                       units = '',
                                       utility_breakpoints=[],
                                       utilities=[],
                                       utility_names=[]
                                       )

TU6_R1 = ISST.Risk(name='', baseline_likelihood = 0.00)
TU6_R2 = ISST.Risk(name='', baseline_likelihood = 0.00)
TU6_R3 = ISST.Risk(name='', baseline_likelihood = 0.00)
TU6_R4 = ISST.Risk(name='', baseline_likelihood = 0.00)
TU6_R5 = ISST.Risk(name='', baseline_likelihood = 0.00)

In [ ]:
TU6_Design_System = ISST.DesignSystem(name='Team Update 6',
                                      model_context=pm.Model(),
                                      risks=[TU6_R1,
                                             TU6_R2,
                                             TU6_R3,
                                             TU6_R4,
                                             TU6_R5],
                                      schedule_risk_table = TU6_Schedule_Impact_Table,
                                      cost_risk_table = TU6_Cost_Impact_Table,
                                      technical_risk_tables = [TU6_T1_Impact_Table,
                                                               TU6_T2_Impact_Table,
                                                               TU6_T3_Impact_Table])

The minimum, maximum, and most likely impacts for each of your risks are the major input variables to keep track of here, so we'll generate a new set of system specification spreadsheets:

In [ ]:
TU6_Design_System.generate_system_specification()

Once you've filled out the spreadsheets (and documented the changes in your individual assignment), we'll read them back in and analyze the results:

In [ ]:
TU6_Design_System.read_system_specification()

In [ ]:
TU6_results = TU6_Design_System.analyze_system()
TU6_results.to_netcdf('TU6_results.nc')

Run this instead if you're coming back later and already have the results:


In [ ]:
TU6_results = az.from_netcdf('TU6_results.nc')

# Comparison with Team Update 5

Now, all that remains is to compare your new results with the baseline from Team Update 5, first, let's plot all of the Total Impact Results for Team Update 5:

In [ ]:
az.plot_posterior(results,
                  var_names=['Total'],
                  filter_vars='like',
                  hdi_prob=0.95)

In [ ]:
az.summary(results,
           var_names=['Total'],
           filter_vars="like",
           round_to=2,
           kind='stats')

And now the same for Team Update 6:

In [ ]:
az.plot_posterior(TU6_results,
                  var_names=['Total'],
                  filter_vars='like',
                  hdi_prob=0.95)

In [ ]:
az.summary(TU6_results,
           var_names=['Total'],
           filter_vars="like",
           round_to=2,
           kind='stats')

And now, let's compare the results in terms of utility:

In [ ]:
TU6_cost_mean = np.asarray(TU6_results.posterior.mean()['Total Cost Impact'])
TU6_schedule_mean = np.asarray(TU6_results.posterior.mean()['Total Schedule Impact'])

TU6_cost_confidence_bounds = np.asarray(az.hdi(TU6_results, hdi_prob=0.95)['Total Cost Impact'])
TU6_schedule_confidence_bounds = np.asarray(az.hdi(TU6_results, hdi_prob=0.95)['Total Schedule Impact'])

If you've changed your utility functions in some way as part of the design revision for Team Update 6, change out the default alias to the old functions below:

In [ ]:
TU6_cost_utility_function = cost_utility_function

In [ ]:
TU6_schedule_utility_function = schedule_utility_function

Now, calculate the change in utility from your design revision:

In [ ]:
Mean_Cost_Utility_Change = TU6_cost_utility_function(TU6_cost_mean) - TU6_cost_utility_function(cost_mean)

LB_Cost_Utility_Change = TU6_cost_utility_function(TU6_cost_confidence_bounds[0]) - TU6_cost_utility_function(cost_confidence_bounds[0])

UB_Cost_Utility_Change = TU6_cost_utility_function(TU6_cost_confidence_bounds[1]) - TU6_cost_utility_function(cost_confidence_bounds[1])

print(f'Mean Cost Utility Change: {Mean_Cost_Utility_Change:.2f}')

print(f'Lower Bound Cost Utility Change: {LB_Cost_Utility_Change:.2f}')

print(f'Upper Bound Cost Utility Change: {UB_Cost_Utility_Change:.2f}')

In [ ]:
Mean_Schedule_Utility_Change = TU6_schedule_utility_function(TU6_schedule_mean) - TU6_schedule_utility_function(schedule_mean)

LB_Schedule_Utility_Change = TU6_schedule_utility_function(TU6_schedule_confidence_bounds[0]) - TU6_schedule_utility_function(schedule_confidence_bounds[0])

UB_Schedule_Utility_Change = TU6_schedule_utility_function(TU6_schedule_confidence_bounds[1]) - TU6_schedule_utility_function(schedule_confidence_bounds[1])

print(f'Mean Cost Utility Change: {Mean_Schedule_Utility_Change:.2f}')

print(f'Lower Bound Cost Utility Change: {LB_Schedule_Utility_Change:.2f}')

print(f'Upper Bound Cost Utility Change: {UB_Schedule_Utility_Change:.2f}')

Now, repeat the process for your technical impacts.

Technical Paramter 1:

In [ ]:
TU6_T1_mean = np.asarray(TU6_results.posterior.mean()[f'Total {TU6_Design_System.techincal_risk_tables[0].name} Impact'])

TU6_T1_confidence_bounds = np.asarray(az.hdi(TU6_results, hdi_prob=0.95)[f'Total {TU6_Design_System.techincal_risk_tables[0].name} Impact'])

In [ ]:
TU6_T1_utility_function = T1_utility_function

In [ ]:
Mean_T1_Utility_Change = TU6_T1_utility_function(TU6_T1_mean) - TU6_T1_utility_function(T1_mean)

LB_T1_Utility_Change = TU6_T1_utility_function(TU6_T1_confidence_bounds[0]) - TU6_T1_utility_function(T1_confidence_bounds[0])

UB_T1_Utility_Change = TU6_T1_utility_function(TU6_T1_confidence_bounds[1]) - TU6_T1_utility_function(T1_confidence_bounds[1])

print(f'Mean Cost Utility Change: {Mean_T1_Utility_Change:.2f}')

print(f'Lower Bound Cost Utility Change: {LB_T1_Utility_Change:.2f}')

print(f'Upper Bound Cost Utility Change: {UB_T1_Utility_Change:.2f}')

Technical Parameter 2:

In [ ]:
TU6_T2_mean = np.asarray(TU6_results.posterior.mean()[f'Total {TU6_Design_System.techincal_risk_tables[1].name} Impact'])

TU6_T2_confidence_bounds = np.asarray(az.hdi(TU6_results, hdi_prob=0.95)[f'Total {TU6_Design_System.techincal_risk_tables[1].name} Impact'])

In [ ]:
TU6_T2_utility_function = T2_utility_function

In [ ]:
Mean_T2_Utility_Change = TU6_T2_utility_function(TU6_T2_mean) - TU6_T2_utility_function(T2_mean)

LB_T2_Utility_Change = TU6_T2_utility_function(TU6_T2_confidence_bounds[0]) - TU6_T2_utility_function(T2_confidence_bounds[0])

UB_T2_Utility_Change = TU6_T2_utility_function(TU6_T2_confidence_bounds[1]) - TU6_T2_utility_function(T2_confidence_bounds[1])

print(f'Mean Cost Utility Change: {Mean_T2_Utility_Change:.2f}')

print(f'Lower Bound Cost Utility Change: {LB_T2_Utility_Change:.2f}')

print(f'Upper Bound Cost Utility Change: {UB_T2_Utility_Change:.2f}')

Technical Parameter 3:

In [ ]:
TU6_T3_mean = np.asarray(TU6_results.posterior.mean()[f'Total {TU6_Design_System.techincal_risk_tables[2].name} Impact'])

TU6_T3_confidence_bounds = np.asarray(az.hdi(TU6_results, hdi_prob=0.95)[f'Total {TU6_Design_System.techincal_risk_tables[2].name} Impact'])

In [ ]:
TU6_T3_utility_function = T3_utility_function

In [ ]:
Mean_T3_Utility_Change = TU6_T3_utility_function(TU6_T3_mean) - TU6_T3_utility_function(T3_mean)

LB_T3_Utility_Change = TU6_T3_utility_function(TU6_T3_confidence_bounds[0]) - TU6_T3_utility_function(T3_confidence_bounds[0])

UB_T3_Utility_Change = TU6_T3_utility_function(TU6_T3_confidence_bounds[1]) - TU6_T3_utility_function(T3_confidence_bounds[1])

print(f'Mean Cost Utility Change: {Mean_T3_Utility_Change:.2f}')

print(f'Lower Bound Cost Utility Change: {LB_T3_Utility_Change:.2f}')

print(f'Upper Bound Cost Utility Change: {UB_T3_Utility_Change:.2f}')

# Assignment Submission:

Once your results are in place for all fo the above elements, print the notebook to a PDF and submit it to Canvas.